# WeRateDogs推特的狗狗的数据收集、评估清洗报告

## 数据整理

### 收集数据

In [2]:
import re
import json
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


plt.rcParams['font.sans-serif'] = ['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False #用来正常显示负号
plt.rcParams['font.family'] = 'serif'
%matplotlib inline

数据集一:读取包含评分、地位、名字等主要推特信息的档案信息

In [3]:
df_twitter_archive_enhanced = pd.read_csv('twitter-archive-enhanced.csv')

数据集二:下载并读取包含推特图像预测信息的文件

In [3]:
url = 'https://raw.githubusercontent.com/udacity/new-dand-advanced-china/master/%E6%95%B0%E6%8D%AE%E6%B8%85%E6%B4%97/WeRateDogs%E9%A1%B9%E7%9B%AE/image-predictions.tsv'
response = requests.get(url)
with open(url.split('/')[-1], 'w') as f:
    f.write(response.text)
df_image_predictions = pd.read_csv(url.split('/')[-1], '\t', encoding="utf8")

数据集三:获取每条推特的额外附加数据

In [4]:
list_tweet = []
with open("tweet_json.txt", "r") as f_tweet:
    for line in f_tweet:
        dict_tweet = json.loads(line)
        tmp = {
            "retweet_count"  : dict_tweet["retweet_count"],
            "favorite_count" : dict_tweet["favorite_count"],
            "tweet_id"       : str(dict_tweet["id_str"])
        }
        list_tweet.append(tmp)
df_tweet_others = pd.DataFrame(list_tweet)

### 评估

#### 目测评估
df_twitter_archive_enhanced表格进行目测评估
- source代表了用户使用什么样的工具来进行访问，其格式不正确，如<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>需要重新进行数据提取
- 存在转发数据需要进行数据删除，如tweet_id为888202515573088257。
- 存在不含图片的数据需要进行删除，如tweet_id为881633300179243008。
- doggo, floofer, pupper和puppo四列代表狗狗的地位应该合并为stage列
- rating_numerator、rating_denominator两列数据代表狗狗的评分，合并为一列rating，并删除这两列数据

In [5]:
df_twitter_archive_enhanced

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None


对df_image_predictions表格进行目测评估
- 预测结果只保留最可能的预测值，将p1, p1_conf,p1_dog,p2, p2_conf,p2_dog,p3, p3_conf,p3_dog合并为p，p_conf, p_dog三列数据

In [6]:
df_image_predictions

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True


In [7]:
df_tweet_others

,favorite_count,retweet_count,tweet_id
0,39492,8842,892420643555336193
1,33786,6480,892177421306343426
2,25445,4301,891815181378084864
3,42863,8925,891689557279858688
4,41016,9721,891327558926688256
5,20548,3240,891087950875897856
6,12053,2142,890971913173991426
7,66596,19548,890729181411237888
8,28187,4403,890609185150312448
9,32467,7684,890240255349198849


<div>
<style>
    .dataframe thead tr:only-child th {
        text-align: right;
    }

#### 编程评估

##### `df_twitter_archive_enhanced`表格

查看数据的基本信息
- tweet_id的数据类型应该更改为str
- timestamp的数据类型应该更改为time
- rating_numerator、rating_denominator应该更改以float
- in_reply_to_status_id、in_reply_to_user_id的数据类型应该更改为str
- retweeted_status_id、retweeted_status_user_id的数据类型应该更改为str
- retweeted_status_timestamp的数据类型应该更改为time

In [8]:
df_twitter_archive_enhanced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [9]:
df_twitter_archive_enhanced.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


rating_numerator，rating_denominator的类型应该更改为float且需要重新进行提取,例如tweet_id为883482846933004288的rating_numerator应该为13.5

In [10]:
item = df_twitter_archive_enhanced[df_twitter_archive_enhanced["tweet_id"] == 883482846933004288]
print(item["text"].values)
print(item["rating_numerator"].values)
print(item["rating_denominator"].values)

['This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10 https://t.co/qjrljjt948']
[5]
[10]


In [11]:
item = df_twitter_archive_enhanced.loc[df_twitter_archive_enhanced["rating_numerator"] == 1776, ]
print(item["tweet_id"].values)
print(item["text"].values)
print(item["rating_numerator"].values)
print(item["rating_denominator"].values)

[749981277374128128]
["This is Atticus. He's quite simply America af. 1776/10 https://t.co/GRXwMxLBkh"]
[1776]
[10]


In [12]:
df_twitter_archive_enhanced.loc[df_twitter_archive_enhanced["rating_numerator"] == 0, ]["text"].values

array(["When you're so blinded by your systematic plagiarism that you forget what day it is. 0/10 https://t.co/YbEJPkg4Ag",
       "PUPDATE: can't see any. Even if I could, I couldn't reach them to pet. 0/10 much disappointment https://t.co/c7WXaB2nqX"],
      dtype=object)

tweet_id为835246439529840640的rating_numerator应该更改为13, rating_denominator更改为10

In [13]:
item = df_twitter_archive_enhanced.loc[df_twitter_archive_enhanced["rating_denominator"] == 0, ]
print(item["tweet_id"])
print(item["text"].values)
print(item["rating_numerator"].values)
print(item["rating_denominator"].values)

313    835246439529840640
Name: tweet_id, dtype: int64
["@jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho"]
[960]
[0]


In [14]:
item = df_twitter_archive_enhanced.loc[df_twitter_archive_enhanced["rating_denominator"] == 170, ]
print(item["tweet_id"])
print(item["text"].values)
print(item["rating_numerator"].values)
print(item["rating_denominator"].values)

1120    731156023742988288
Name: tweet_id, dtype: int64
['Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv']
[204]
[170]


检查tweet_id是否存在重复值,无重复值

In [15]:
df_twitter_archive_enhanced["tweet_id"].duplicated().sum()

0

查看name的唯一值情况

In [16]:
df_twitter_archive_enhanced["name"].unique()

array(['Phineas', 'Tilly', 'Archie', 'Darla', 'Franklin', 'None', 'Jax',
       'Zoey', 'Cassie', 'Koda', 'Bruno', 'Ted', 'Stuart', 'Oliver',
       'Jim', 'Zeke', 'Ralphus', 'Canela', 'Gerald', 'Jeffrey', 'such',
       'Maya', 'Mingus', 'Derek', 'Roscoe', 'Waffles', 'Jimbo', 'Maisey',
       'Lilly', 'Earl', 'Lola', 'Kevin', 'Yogi', 'Noah', 'Bella',
       'Grizzwald', 'Rusty', 'Gus', 'Stanley', 'Alfy', 'Koko', 'Rey',
       'Gary', 'a', 'Elliot', 'Louis', 'Jesse', 'Romeo', 'Bailey',
       'Duddles', 'Jack', 'Emmy', 'Steven', 'Beau', 'Snoopy', 'Shadow',
       'Terrance', 'Aja', 'Penny', 'Dante', 'Nelly', 'Ginger', 'Benedict',
       'Venti', 'Goose', 'Nugget', 'Cash', 'Coco', 'Jed', 'Sebastian',
       'Walter', 'Sierra', 'Monkey', 'Harry', 'Kody', 'Lassie', 'Rover',
       'Napolean', 'Dawn', 'Boomer', 'Cody', 'Rumble', 'Clifford',
       'quite', 'Dewey', 'Scout', 'Gizmo', 'Cooper', 'Harold', 'Shikha',
       'Jamesy', 'Lili', 'Sammy', 'Meatball', 'Paisley', 'Albus',
       'Nept

- 存在非狗类的数据，需要进行删除，筛选text中包含We only rate dogs, Dogs are what we rate, It is not a dog,I only rate dogs,Only send in dogs.的数据
- 存在name为a, an, None提取错误，如tweet_id为667524857454854144，提取H(h)is name is, named,否则name更改为np.nan。

In [17]:
df_twitter_archive_enhanced[df_twitter_archive_enhanced["name"] == "a"]["text"].values

array(['Here is a pupper approaching maximum borkdrive. Zooming at never before seen speeds. 14/10 paw-inspiring af \r\n(IG: puffie_the_chow) https://t.co/ghXBIIeQZF',
       'Here is a perfect example of someone who has their priorities in order. 13/10 for both owner and Forrest https://t.co/LRyMrU7Wfq',
       'Guys this is getting so out of hand. We only rate dogs. This is a Galapagos Speed Panda. Pls only send dogs... 10/10 https://t.co/8lpAGaZRFn',
       'This is a mighty rare blue-tailed hammer sherk. Human almost lost a limb trying to take these. Be careful guys. 8/10 https://t.co/TGenMeXreW',
       'Viewer discretion is advised. This is a terrible attack in progress. Not even in water (tragic af). 4/10 bad sherk https://t.co/L3U0j14N5R',
       'This is a carrot. We only rate dogs. Please only send in dogs. You all really should know this by now ...11/10 https://t.co/9e48aPrBm2',
       'This is a very rare Great Alaskan Bush Pupper. Hard to stumble upon without spooking. 12/

In [68]:
df_twitter_archive_enhanced[df_twitter_archive_enhanced["name"] == "an"]["text"].values

array(['RT @dog_rates: This is an East African Chalupa Seal. We only rate dogs. Please only send in dogs. Thank you... 10/10 https://t.co/iHe6liLwWR',
       "This is an Iraqi Speed Kangaroo. It is not a dog. Please only send in dogs. I'm very angry with all of you ...9/10 https://t.co/5qpBTTpgUt",
       'This is an East African Chalupa Seal. We only rate dogs. Please only send in dogs. Thank you... 10/10 https://t.co/iHe6liLwWR',
       'This is an Irish Rigatoni terrier named Berta. Completely made of rope. No eyes. Quite large. Loves to dance. 10/10 https://t.co/EM5fDykrJg',
       'This is an extremely rare horned Parthenon. Not amused. Wears shoes. Overall very nice. 9/10 would pet aggressively https://t.co/QpRjllzWAL',
       'This is an Albanian 3 1/2 legged  Episcopalian. Loves well-polished hardwood flooring. Penis on the collar. 9/10 https://t.co/d9NcXFKwLv',
       "This is an odd dog. Hard on the outside but loving on the inside. Petting still fun. Doesn't play catch well.

In [69]:
df_twitter_archive_enhanced[df_twitter_archive_enhanced["name"] == "None"]["text"].values

array(["Here we have a majestic great white breaching off South Africa's coast. Absolutely h*ckin breathtaking. 13/10 (IG: tucker_marlo) #BarkWeek https://t.co/kQ04fDDRmh",
       "When you watch your owner call another dog a good boy but then they turn back to you and say you're a great boy. 13/10 https://t.co/v0nONBcwxq",
       "Here's a puppo that seems to be on the fence about something haha no but seriously someone help her. 13/10 https://t.co/BxvuXk0UCm",
       'You may not have known you needed to see this today. 13/10 please enjoy (IG: emmylouroo) https://t.co/WZqNqygEyV',
       'This... is a Jubilant Antarctic House Bear. We only rate dogs. Please only send dogs. Thank you... 12/10 would suffocate in floof https://t.co/4Ad1jzJSdp',
       '@NonWhiteHat @MayhewMayhem omg hello tanner you are a scary good boy 12/10 would pet with extreme caution',
       'RT @Athletics: 12/10 #BATP https://t.co/WxwJmvjfxo',
       'I have a new hero and his name is Howard. 14/10 https://t.co/

In [18]:
df_twitter_archive_enhanced.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'source', 'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name', 'doggo', 'floofer', 'pupper', 'puppo'],
      dtype='object')

`df_twitter_archive_enhanced`各列代表的意义:
- tweet_id: twitter ID
- in_reply_to_status_id:
- in_reply_to_user_id
- timestamp:时间戳
- source:用户访问的设备
- text:推特原文
- retweeted_status_id
- retweeted_status_user_id:
- retweeted_status_timestamp:
- expanded_urls:
- rating_numerator:评分的分母
- rating_denominator:评分的分子
- name:狗的名字
- doggo: 狗的地位
- floofer:狗的地位
- pupper: 狗的地位
- puppo: 狗的地位

##### `df_image_predictions`表格
- tweet_id的数据类型更改为str.

In [19]:
df_image_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [20]:
df_image_predictions.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


检查tweet_id无重复值

In [21]:
df_image_predictions["tweet_id"].duplicated().sum()

0

In [22]:
df_image_predictions.columns

Index(['tweet_id', 'jpg_url', 'img_num', 'p1', 'p1_conf', 'p1_dog', 'p2',
       'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog'],
      dtype='object')

df_image_predictions各列数据代表的意义:
- tweet_id:推特链接的最后一部分，位于 "status/" 后面 → https://twitter.com/dog_rates/status/889531135344209921
- jpg_url 是预测的图像资源链接
- img_num 最可信的预测结果对应的图像编号 → 1 推特中的第一张图片
- p1 是算法对推特中图片的一号预测 → 金毛犬
- p1_conf 是算法的一号预测的可信度 → 95%
- p1_dog 是一号预测该图片是否属于“狗”（有可能是其他物种，比如熊、马等） → True 真
- p2 是算法对推特中图片预测的第二种可能性 → 拉布拉多犬
- p2_conf 是算法的二号预测的可信度 → 1%
- p2_dog 是二号预测该图片是否属于“狗” → True 真
- p3 是算法对推特中图片预测的第三种可能性
- p3_conf 是算法的三号预测的可信度 → 1%
- p3_dog 是三号预测该图片是否属于“狗” → True 真

##### `df_tweet_others`表格

In [26]:
df_tweet_others.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2352 entries, 0 to 2351
Data columns (total 3 columns):
favorite_count    2352 non-null int64
retweet_count     2352 non-null int64
tweet_id          2352 non-null object
dtypes: int64(2), object(1)
memory usage: 55.2+ KB


In [61]:
df_tweet_others.describe()

,favorite_count,retweet_count
count,2352.000000,2352.000000
mean,8109.198980,3134.932398
std,11980.795669,5237.846296
min,0.000000,0.000000
25%,1417.000000,618.000000
50%,3596.500000,1456.500000
75%,10118.000000,3628.750000
max,132318.000000,79116.000000


In [20]:
df_tweet_others.columns

Index(['favorite_count', 'retweet_count', 'tweet_id'], dtype='object')

`df_tweet_others`表格各列数据代表的意义:
- retweet_count:转发数
- favorite_count:喜爱数
- tweet_id:twitter ID

##### 质量
###### `df_twitter_archive_enhanced`表格
- source代表了用户使用什么样的工具来进行访问，其格式不正确，如<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>需要重新进行数据提取
- 存在转发数据需要进行数据删除，如tweet_id为888202515573088257。
    - 删除retweeted_status_id不为空的数据
    - text中含RT @的数据
- 存在不含图片的数据需要进行删除，如tweet_id为881633300179243008。
    - df_tweet_others的jpg_url代表图片资源，将这两个表格进行内连接
- in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp对狗狗的评分无意义删除相关列数据。
- tweet_id、timestamp、retweeted_status_id、retweeted_status_user_id、retweeted_status_timestamp的数据类型错误，分别更改为str、datetime、str、str、datetime
- rating_numerator、rating_denominator数据不准确，需要重新进行提取,例如tweet_id为883482846933004288的rating_numerator应该为13.5,且数据类型需要更改为float。
- tweet_id为835246439529840640的rating_numerator应该更改为13, rating_denominator更改为10
- 存在非狗类的数据，需要进行删除，筛选text中包含We only rate dogs, Dogs are what we rate, It is not a dog的数据
- 存在name为a, an, None提取错误，如tweet_id为667524857454854144，提取H(h)is name is, named,否则name更改为np.nan。
- 存在转发数和喜爱数位于df_tweet_others表格中，数据不完整，需要将df_image_predictions、df_tweet_others、df_twitter_archive_enhanced三个表格合并为一个表格。
###### `df_image_predictions` 表格
- tweet_id的数据类型更改为str.

##### 整洁度
###### `df_twitter_archive_enhanced`表格
- doggo, floofer, pupper和puppo四列代表狗狗的地位应该合并为stage列并删除这四列数据。
- rating_numerator、rating_denominator两列数据代表狗狗的评分，合并为一列rating，并删除这两列数据

###### `df_image_predictions` 表格
- 预测结果只保留最可能的预测值，将p1, p1_conf,p1_dog,p2, p2_conf,p2_dog,p3, p3_conf,p3_dog合并为p，p_conf, p_dog三列数据

###### `df_tweet_others` 表格
- df_tweet_others表格都是关于狗狗评分的数据表，将df_tweet_others、df_image_predictions、df_twitter_archive_enhanced合并为一个表格.

### 清理

清洗前，进行数据备份

In [541]:
df_cleaned_twitter_archive_enhanced = df_twitter_archive_enhanced.copy()
df_cleaned_image_predictions = df_image_predictions.copy()
df_cleaned_tweet_others = df_tweet_others.copy()

#### 问题定义1：更改数据类型错误问题
- df_twitter_archive_enhanced表格中tweet_id、timestamp、retweeted_status_id、retweeted_status_user_id、retweeted_status_timestamp的数据类型错误，分别更改为str、datetime、str、str、datetime
- df_image_predictions表格中tweet_id的数据类型更改为str.

#### 代码

In [542]:
df_cleaned_twitter_archive_enhanced["tweet_id"] = df_cleaned_twitter_archive_enhanced["tweet_id"].astype(str)
df_cleaned_twitter_archive_enhanced["timestamp"] = pd.to_datetime(df_cleaned_twitter_archive_enhanced["timestamp"])
df_cleaned_twitter_archive_enhanced["retweeted_status_id"] = df_cleaned_twitter_archive_enhanced[df_cleaned_twitter_archive_enhanced["retweeted_status_id"].notna()]["retweeted_status_id"].astype(str)
df_cleaned_twitter_archive_enhanced["retweeted_status_timestamp"] = \
               pd.to_datetime(df_cleaned_twitter_archive_enhanced["retweeted_status_timestamp"])
df_cleaned_image_predictions["tweet_id"] = df_cleaned_image_predictions["tweet_id"].astype(str)

#### 测试

In [543]:
df_cleaned_twitter_archive_enhanced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null object
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null datetime64[ns]
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null object
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null datetime64[ns]
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: datetime

In [544]:
df_cleaned_image_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null object
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(1), object(5)
memory usage: 152.1+ KB


#### 问题定义2：`df_image_predictions` 表格中预测结果只保留最可能的预测值
- 将p1, p1_conf,p1_dog,p2, p2_conf,p2_dog,p3, p3_conf,p3_dog合并为p，p_conf, p_dog三列数据

#### 代码

In [545]:
df_cleaned_image_predictions["p"] = np.nan
df_cleaned_image_predictions["p_conf"] = np.nan
df_cleaned_image_predictions["p_dog"] = np.nan
for index, row in df_cleaned_image_predictions.iterrows():
    name = "p" + str(row["img_num"])
    if row["img_num"] <= 3:
        df_cleaned_image_predictions.loc[ index, "p"] = row["p" + str(row["img_num"])]
        df_cleaned_image_predictions.loc[ index, "p_conf"] = row["p" + str(row["img_num"]) + "_conf"]
        df_cleaned_image_predictions.loc[ index, "p_dog"] = row["p" + str(row["img_num"]) + "_dog"]
df_cleaned_image_predictions.drop(["p1", "p1_conf", "p1_dog", "p2",  "p2_conf", "p2_dog", \
                                   "p3", "p3_conf", "p3_dog"], axis = 1, inplace=True)

#### 测试

In [546]:
df_cleaned_image_predictions.head(10)

,tweet_id,jpg_url,img_num,p,p_conf,p_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True


#### 问题定义3 : 存在转发数据需要进行数据删除，如tweet_id为888202515573088257。
- 删除retweeted_status_id不为空的数据
- text中含RT @的数据

#### 代码

In [547]:
indies_retweeted_not_na = df_cleaned_twitter_archive_enhanced[df_cleaned_twitter_archive_enhanced["retweeted_status_id"].notna()].index.tolist()
df_cleaned_twitter_archive_enhanced.drop(indies_retweeted_not_na, axis=0, inplace=True)

indexs_rt = df_cleaned_twitter_archive_enhanced[df_cleaned_twitter_archive_enhanced["text"].str.contains("RT@")].index.tolist()
df_cleaned_twitter_archive_enhanced.drop(indexs_rt, axis=0, inplace=True)

#### 测试

In [548]:
df_cleaned_twitter_archive_enhanced["retweeted_status_id"].notna().sum()

0

In [549]:
df_cleaned_twitter_archive_enhanced["text"].str.contains("RT @").sum()

0

#### 问题定义4：- in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp对狗狗的评分无意义删除相关列数据。

##### 代码

In [550]:
df_cleaned_twitter_archive_enhanced.drop(["in_reply_to_status_id", "in_reply_to_user_id",
            "retweeted_status_id", "retweeted_status_user_id", "retweeted_status_timestamp"], inplace = True, axis = 1)

##### 测试

In [551]:
df_cleaned_twitter_archive_enhanced.columns

Index(['tweet_id', 'timestamp', 'source', 'text', 'expanded_urls',
       'rating_numerator', 'rating_denominator', 'name', 'doggo', 'floofer',
       'pupper', 'puppo'],
      dtype='object')

#### 问题定义5： 存在不含图片的数据需要进行删除，如tweet_id为881633300179243008。
- df_cleaned_image_predictions的jpg_url代表图片资源，将这两个表格进行内连接

#### 代码

In [552]:
df_cleaned_twitter_archive_enhanced = pd.merge(df_cleaned_twitter_archive_enhanced, df_cleaned_image_predictions, \
                                               how="inner", on = "tweet_id")

#### 测试

In [553]:
df_cleaned_twitter_archive_enhanced.head()

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,jpg_url,img_num,p,p_conf,p_dog
0,892420643555336193,2017-08-01 16:23:56,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,1,orange,0.097049,False
1,892177421306343426,2017-08-01 00:17:27,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1,Chihuahua,0.323581,True
2,891815181378084864,2017-07-31 00:18:03,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,0.716012,True
3,891689557279858688,2017-07-30 15:58:51,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,paper_towel,0.170278,False
4,891327558926688256,2017-07-29 16:00:24,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2,English_springer,0.225770,True


#### 问题6：df_tweet_others表格都是关于狗狗评分的数据表，将df_tweet_others、df_image_predictions、df_twitter_archive_enhanced合并为一个表格.

#### 代码

In [555]:
df_cleaned_twitter_archive_enhanced = pd.merge(df_cleaned_twitter_archive_enhanced, \
                                               df_cleaned_tweet_others, how="left", on = "tweet_id")

#### 测试

In [557]:
df_cleaned_twitter_archive_enhanced.head()

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,jpg_url,img_num,p,p_conf,p_dog,favorite_count,retweet_count
0,892420643555336193,2017-08-01 16:23:56,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,1,orange,0.097049,False,39492,8842
1,892177421306343426,2017-08-01 00:17:27,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1,Chihuahua,0.323581,True,33786,6480
2,891815181378084864,2017-07-31 00:18:03,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,0.716012,True,25445,4301
3,891689557279858688,2017-07-30 15:58:51,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,paper_towel,0.170278,False,42863,8925
4,891327558926688256,2017-07-29 16:00:24,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2,English_springer,0.225770,True,41016,9721


#### 问题定义7: rating_numerator、rating_denominator数据不准确，需要重新进行提取,例如tweet_id为883482846933004288的rating_numerator应该为13.5,且数据类型需要更改为float
- 重新提取rating_numerator、rating_denominator数据

#### 问题6定义: 将rating_numerator、rating_denominator合并为一列rating，删除rating_numerator、rating_denominator。

#### 代码

In [558]:
rating = df_cleaned_twitter_archive_enhanced["text"].str.lower().str.extract(r"((?:\d+\.)?\d+)\/(\d+)", expand=True)
rating.columns = ["rating_numerator", "rating_denominator"]
df_cleaned_twitter_archive_enhanced["rating"] = rating["rating_numerator"].astype(float) / rating["rating_denominator"].astype(float)
df_cleaned_twitter_archive_enhanced.drop(["rating_numerator", "rating_denominator"],axis=1, inplace=True)

#### 测试

In [559]:
df_cleaned_twitter_archive_enhanced.head()

,tweet_id,timestamp,source,text,expanded_urls,name,doggo,floofer,pupper,puppo,jpg_url,img_num,p,p_conf,p_dog,favorite_count,retweet_count,rating
0,892420643555336193,2017-08-01 16:23:56,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,Phineas,None,None,None,None,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,1,orange,0.097049,False,39492,8842,1.3
1,892177421306343426,2017-08-01 00:17:27,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,Tilly,None,None,None,None,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1,Chihuahua,0.323581,True,33786,6480,1.3
2,891815181378084864,2017-07-31 00:18:03,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,https://twitter.com/dog_rates/status/891815181...,Archie,None,None,None,None,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,0.716012,True,25445,4301,1.2
3,891689557279858688,2017-07-30 15:58:51,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,https://twitter.com/dog_rates/status/891689557...,Darla,None,None,None,None,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,paper_towel,0.170278,False,42863,8925,1.3
4,891327558926688256,2017-07-29 16:00:24,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,https://twitter.com/dog_rates/status/891327558...,Franklin,None,None,None,None,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2,English_springer,0.225770,True,41016,9721,1.2


In [560]:
df_cleaned_twitter_archive_enhanced.columns

Index(['tweet_id', 'timestamp', 'source', 'text', 'expanded_urls', 'name',
       'doggo', 'floofer', 'pupper', 'puppo', 'jpg_url', 'img_num', 'p',
       'p_conf', 'p_dog', 'favorite_count', 'retweet_count', 'rating'],
      dtype='object')

#### 问题定义8:doggo, floofer, pupper和puppo四列代表狗狗的地位应该合并为stage列并删除这四列数据。
- tweets记录包含不止一个stage值的情况，保留每一个出现的stage值，同时不同的值之间用符号隔开

#### 代码

In [561]:
# 提取包含地位相关的字段
df_cleaned_twitter_archive_enhanced["stage"] = df_cleaned_twitter_archive_enhanced["text"].str.lower()\
        .str.findall(r'(doggo|floof|pupper|puppo)')
# 去重
df_cleaned_twitter_archive_enhanced["stage"] = df_cleaned_twitter_archive_enhanced["stage"].apply(lambda x:set(x))
df_cleaned_twitter_archive_enhanced["stage"] = df_cleaned_twitter_archive_enhanced["stage"].apply(lambda x:";".join(x))
df_cleaned_twitter_archive_enhanced["stage"] = df_cleaned_twitter_archive_enhanced["stage"].replace("", np.nan)
df_cleaned_twitter_archive_enhanced.drop(["doggo", "floofer", "pupper", "puppo"], axis=1, inplace=True)

#### 测试

In [562]:
df_cleaned_twitter_archive_enhanced["stage"].unique()

array([nan, 'doggo', 'puppo', 'floof', 'pupper', 'doggo;puppo',
       'doggo;floof', 'doggo;pupper', 'pupper;floof'], dtype=object)

In [563]:
df_cleaned_twitter_archive_enhanced.columns

Index(['tweet_id', 'timestamp', 'source', 'text', 'expanded_urls', 'name',
       'jpg_url', 'img_num', 'p', 'p_conf', 'p_dog', 'favorite_count',
       'retweet_count', 'rating', 'stage'],
      dtype='object')

#### 问题9: tweet_id为835246439529840640的rating_numerator应该更改为13, rating_denominator更改为10
    - text中包含**/** isn't a valid rating
#### 测试
tweet_id为835246439529840640的数据已经被清除

In [564]:
df_cleaned_twitter_archive_enhanced[df_cleaned_twitter_archive_enhanced["tweet_id"] == "835246439529840640"]

,tweet_id,timestamp,source,text,expanded_urls,name,jpg_url,img_num,p,p_conf,p_dog,favorite_count,retweet_count,rating,stage


#### 问题定义10: 存在非狗类的数据，需要进行删除
- 筛选text中包含We only rate dogs, Dogs are what we rate, It is not a dog的数据

#### 代码

In [565]:
not_dog_index_list = df_cleaned_twitter_archive_enhanced[df_cleaned_twitter_archive_enhanced.text.str.lower().str \
                                                         .contains("we only rate dogs")].index.tolist()
not_dog_index_list.extend(df_cleaned_twitter_archive_enhanced[df_cleaned_twitter_archive_enhanced.text.str.lower().str \
                                                         .contains("dogs are what we rate")].index.tolist())
not_dog_index_list.extend(df_cleaned_twitter_archive_enhanced[df_cleaned_twitter_archive_enhanced.text.str.lower().str \
                                                         .contains("it is not a dog")].index.tolist())
# 去重
not_dog_index_list = list(set(not_dog_index_list))
df_cleaned_twitter_archive_enhanced.drop(labels=not_dog_index_list, axis=0, inplace=True)

#### 测试

In [566]:
test_not_dog_index_list =df_cleaned_twitter_archive_enhanced[df_cleaned_twitter_archive_enhanced.text.str.lower().str.contains("we only rate dogs")].index.tolist()
test_not_dog_index_list.extend(df_cleaned_twitter_archive_enhanced[df_cleaned_twitter_archive_enhanced.text.str.lower().str.contains("dogs are what we rate")].index.tolist())
test_not_dog_index_list.extend(df_cleaned_twitter_archive_enhanced[df_cleaned_twitter_archive_enhanced.text.str.lower().str.contains("it is not a dog")].index.tolist())
test_not_dog_index_list

[]

#### 问题11: 存在name为a, an, None提取错误，如tweet_id为667524857454854144
- 提取H(h)is name is之后的字符
- 提取包含named之后的字符
- 提取包含H(h)er name is之后的字符
- 不包含上述字符，否则name更改为""。
#### 代码

In [567]:
def get_real_name(regex,  s):
    matches = re.finditer(regex, s, re.MULTILINE)
    for matchNum, match in enumerate(matches, start=1):
        return match.group(len(match.groups()))
for index, row in df_cleaned_twitter_archive_enhanced.iterrows():
    if row["name"].lower() == "a" or row["name"].lower() == "an" or row["name"].lower() == "none":
        if "his name is" in row["text"].lower():
            real_name = get_real_name(r"his\sname\sis\s(\w+)", row["text"].lower()).title()
            df_cleaned_twitter_archive_enhanced.loc[index, "name"] = real_name
        else:
            if "her name is" in row["text"].lower():
                real_name =get_real_name(r"her\sname\sis\s(\w+)", row["text"].lower()).title()
                df_cleaned_twitter_archive_enhanced.loc[index, "name"] = real_name
            else:
                if "named" in row["text"].lower():
                    real_name = get_real_name(r"named\s(\w+)" ,row["text"].lower())
                    df_cleaned_twitter_archive_enhanced.loc[index, "name"] = real_name
                else:
                    df_cleaned_twitter_archive_enhanced.loc[index, "name"] = ""

#### 测试

In [568]:
df_cleaned_twitter_archive_enhanced[df_cleaned_twitter_archive_enhanced["name"] == "a"]
df_cleaned_twitter_archive_enhanced[df_cleaned_twitter_archive_enhanced["name"] == "an"]
df_cleaned_twitter_archive_enhanced[df_cleaned_twitter_archive_enhanced["name"] == "none"]

,tweet_id,timestamp,source,text,expanded_urls,name,jpg_url,img_num,p,p_conf,p_dog,favorite_count,retweet_count,rating,stage


#### 问题定义12: `df_twitter_archive_enhanced`表格source代表了用户使用什么样的工具来进行访问，其格式不正确，重新进行数据提取
#### 代码

In [569]:
df_cleaned_twitter_archive_enhanced["source"] = df_cleaned_twitter_archive_enhanced["source"].str.extract(r">(.*)</a>", expand=True)

##### 测试

In [570]:
df_cleaned_twitter_archive_enhanced["source"].unique()

array(['Twitter for iPhone', 'Twitter Web Client', 'TweetDeck'],
      dtype=object)

### 存储清理后的主数据集

In [571]:
# 将清理后的主数据集保存为 csv 文件
df_cleaned_twitter_archive_enhanced.to_csv("twitter_archive_master.csv", index=False)